In [14]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

# import sys
# sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

1027

In [4]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))

    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数

    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

will use cuda


In [5]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [6]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)


In [7]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [8]:
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 151.075481, time 0.23 sec
 - 分开 我不的让我爱爱女人 坏坏的让我爱女人 坏坏的让我爱女人 坏坏的让我爱女人 坏坏的让我爱女人 坏坏的
 - 不分开 我不要你的可爱女人人 坏坏的让我爱爱女人 坏坏的让我爱女人 坏坏的让我爱女人 坏坏的让我爱女人 坏
epoch 80, perplexity 33.953324, time 0.23 sec
 - 分开 我想要这样 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 - 不分开 不知不觉 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不
epoch 120, perplexity 5.870606, time 0.23 sec
 - 分开 一直走 我想你这样牵着你的手不放  爱后风 后实我早开头 一直走秋 你在那空 在一场空 在人的钟 
 - 不分开 你已经离是我 不知不觉 我跟了这节奏 后知后觉 我该好好生活 我该好好生活 又静悄悄默默 后在开不
epoch 160, perplexity 1.797464, time 0.24 sec
 - 分开 一个我 已过我 爱是我 没是是没有 有有直在 全的梦空 不变就痛  没什么 瞎不上 恨不懂 从漠是
 - 不分开 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活


In [9]:
lr = 1e-2 # 注意调整学习率
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.014966, time 0.04 sec
 - 分开球 你在操纵 恨自己真的没用 情绪激动 一颗心到现在还在抽痛 还为分手前那句抱歉 在感动 穿梭时间的
 - 不分开始打呼 管家是一只会说法语举止优雅的猪 吸血前会念约翰福音做为弥补 拥有一双蓝色眼睛的凯萨琳公主 专
epoch 80, perplexity 1.011363, time 0.06 sec
 - 分开球我的肩膀 你 在我胸口睡著 像这样的生活 我爱你 你爱我 我想大声宣布 对你依依不舍 连隔壁邻居都
 - 不分开始 担心今天的你过得好不好 整个画面是你 想你想的睡不著 嘴嘟嘟那可爱的模样 还有在你身上香香的味道
epoch 120, perplexity 1.010964, time 0.04 sec
 - 分开球 懂不懂篮球 有种不要走 三对三斗牛 三分球 它在空中停留 所有人看着我 抛物线进球 单手过人运球
 - 不分开始 担心今天的你过得好不好 整个画面是你 想你想的睡不著 嘴嘟嘟那可爱的模样 还有在你身上香香的味道
epoch 160, perplexity 1.362794, time 0.04 sec
 - 分开的我有一双翅膀 二双翅膀 随时出来一直到我们都睡着 我想就这样牵着你的手不放开 爱像一阵风 吹完它一
 - 不分开不 我不要再想三 有何不同 你不懂 连棍棒 一颗两颗三颗四颗 连成线背著背默默许下心愿 看远方的星是
